In [22]:
import numpy as np
import torch
import yaml
import importlib
import random

# So that we can plot sisl geometries
import sisl.viz

from e3nn import o3
from pathlib import Path

from graph2mat import (
    PointBasis,
    BasisTableWithEdges,
    BasisConfiguration,
    MatrixDataProcessor,
)
from graph2mat.bindings.torch import TorchBasisMatrixData, TorchBasisMatrixDataset
from graph2mat.bindings.e3nn import E3nnGraph2Mat

def load_config(path="../config.yaml"):
    with open(path, "r") as f:
        return yaml.safe_load(f)
    
def flatten(xss):
    return [x for xs in xss for x in xs]

def is_orthogonal(matrix):
    """
    Check if a matrix is diagonal.
    """
    product = np.dot(matrix, matrix.T)
    nz = np.count_nonzero(product - np.diag(np.diagonal(product)))
    if nz == 0:
        return True
    else:
        return False 

In [ ]:
# === List of paths to all structures ===
parent_path = Path('../dataset')
n_atoms_paths = list(parent_path.glob('*/'))
paths = []
for n_atoms_path in n_atoms_paths:
    structure_paths = list(n_atoms_path.glob('*/'))
    paths.append(structure_paths)
paths = flatten(paths)

random.seed(42)
random.shuffle(paths)

# === Find a non-orthogonal structure ===
for path in paths:
    file = sisl.get_sile(path / "aiida.fdf")
    geometry = file.read_geometry()
    cell = geometry.lattice.cell
    non_orth_cell = cell if not is_orthogonal(cell) else None
    if non_orth_cell is not None:
        print(f"Found a non-orthogonal cell at {path}")
        print(non_orth_cell)
        file.plot.geometry(axes="xz")
        break
if non_orth_cell is None:
    print("No non-orthogonal cell found in the dataset.")



No non-orthogonal cell found in the dataset.
